## 3) Group comparison between older and younger adults

- Each EEG feature is compared between younger and older adults using non-parametric Mann-Whitney tests
- The p-values are corrected for multiple comparisons using FDR
- Effect sizes are presented as ' r values ' obtained using the function 'wilcox_effsize' 

Gordillo, da Cruz, Moreno, Garobbio, Herzog

In [ ]:
#rm(list=ls(all=TRUE))  # clear all variables
# dario.gordillolopez@epfl.ch
library('coin')
library('rstatix')

In [ ]:
# rm(list=ls(all=TRUE))  # clear all variables
set.seed(42) # set seed for p-values using coin
setwd(getwd())
eeg_data_dir = file.path(getwd(),"data","csv_data")

In [ ]:
# Find EEG files in .csv 
temp <- list.files(path = eeg_data_dir ,pattern = "*.csv")

In [ ]:
# to store data
pvalues <- vector()
z_stat <- vector() 
selected_ch <- vector()
r_stat <- vector()
rcil_stat <- vector()
rcih_stat <- vector()

In [ ]:
# mwu for each eeg feature
for (i in 1:length(temp)){
  # load EEG data
  data_EEG <- read.csv(file.path(eeg_data_dir, temp[i]))
  data_EEG$Group <- factor(data_EEG$Group)  
  variables_EEG <- colnames(data_EEG)[5:ncol(data_EEG)]
  # internal store
  MAIN_p_values <- vector()
  MAIN_z <- vector()
  MAIN_fx_r <- vector()
  MAIN_fx_r_cilow <- vector()
  MAIN_fx_r_cihigh <- vector()  
    
  for (j in 1:length(variables_EEG)){
    # create data frame
    eeg_var <- data_EEG[, j+4]    
    data_model <- data.frame(eeg_data = I(eeg_var), Group = I(data_EEG[,3]))
    # statistical test
    mwu_test <- coin::wilcox_test(eeg_var ~ Group, data = data_model)
    fx_test <-  wilcox_effsize(eeg_data ~ Group, data = data_model, ci = "TRUE", ci.type="perc")

    # store results
    MAIN_p_values[j] <- pvalue(mwu_test)
    MAIN_z[j] <- statistic(mwu_test)
    MAIN_fx_r[j] <- as.numeric(fx_test$effsize)
    MAIN_fx_r_cilow[j] <- fx_test$conf.low
    MAIN_fx_r_cihigh[j] <- fx_test$conf.high
    
    message('Processing variables ', j, ' of ', length(variables_EEG))
    flush.console()
  }
  
  min_p_val <- which.min(MAIN_p_values)
  adjust_p_values <- p.adjust(MAIN_p_values, method = "BH")
  
  # all data
  selected_ch[i] <- min_p_val
  pvalues[i] <- adjust_p_values[min_p_val]
  z_stat[i] <- MAIN_z[min_p_val] 
  r_stat[i] <- MAIN_fx_r[min_p_val] 
  rcil_stat[i] <- MAIN_fx_r_cilow[min_p_val] 
  rcih_stat[i] <- MAIN_fx_r_cihigh[min_p_val] 
  
  message('Processing features ', i, ' of ', length(temp))
}



In [ ]:
cols_mwu <- cbind(selected_ch, pvalues, z_stat, r_stat, rcil_stat, rcih_stat)
cols_mwu <- as.data.frame(cbind(temp,cols_mwu))
colnames(cols_mwu)[which(names(cols_mwu) == "temp")] <- "features"
save_ <- file.path(getwd(),"results", "3_group_comparison_results", "3_mwu_r.csv")
write.csv(cols_mwu,save_, row.names = FALSE)